In [5]:
import torch
from torch.utils.cpp_extension import load
from algo import original_orth
import timeit

In [7]:
qr_func = load(name="qr_func", sources=["qr_orthogonalization.cpp", "qr_orthogonalization.cu"])

In [19]:
M = 8
N = 1024
device = torch.device("cuda")
dtype = torch.float16

A = torch.rand((M, N), device=device, dtype=dtype)
Q = torch.zeros((M,N), device=device, dtype=dtype)

In [20]:
qr_func.qr_orthogonalization(A.clone(), Q, 0)

In [21]:
torch.norm(Q @ Q.T - torch.eye(M, device = device, dtype=dtype))

RuntimeError: BFloat16 gemm in CUDA requires Ampere or later GPU

In [28]:
Q_or = original_orth(A.clone(), 0)

In [101]:
torch.norm(Q - Q_or)

tensor(0.2016, device='cuda:0')

In [102]:
torch.norm(Q_or @ Q_or.T - torch.eye(M, device = device))

tensor(6.7086e-07, device='cuda:0')

In [103]:
%timeit torch.cuda.synchronize(); qr_func.qr_orthogonalization(A.clone(), Q, 0); torch.cuda.synchronize();

418 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [104]:
%timeit torch.cuda.synchronize(); torch.qr(A.clone()); torch.cuda.synchronize();

10.2 ms ± 318 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
